In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import kagglehub
import os

In [38]:
path = kagglehub.dataset_download("yasserh/housing-prices-dataset")
df = pd.read_csv(os.path.join(path, "Housing.csv"), header=0)
print(path)

C:\Users\gusta\.cache\kagglehub\datasets\yasserh\housing-prices-dataset\versions\1


In [39]:
NUMERICAL_FEATURES = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
BINARY_FEATURES = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
NOMINAL_FEATURES = ['furnishingstatus']
TARGET = 'price'

In [40]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [41]:

CATEGORICAL_FEATURES = BINARY_FEATURES + NOMINAL_FEATURES

## 1. Data Structure and Types (df.info())
print("1. DATA STRUCTURE AND NON-NULL COUNTS")
df.info()

# ---

## 2. Descriptive Statistics (df.describe())
print("2. DESCRIPTIVE STATISTICS (Numerical Features)")
print(df[NUMERICAL_FEATURES].describe().T)

# ---

## 3. Missing Values Check (df.isnull().sum())
print("3. MISSING VALUES COUNT")
print(df.isnull().sum())

# ---

## 4. Categorical Feature Analysis (Value Counts)
print("4. CATEGORICAL FEATURE VALUE COUNTS")
for col in CATEGORICAL_FEATURES:
    print(f"\n--- {col.upper()} ---")
    print(df[col].value_counts())

1. DATA STRUCTURE AND NON-NULL COUNTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB
2. DESCRIPTIVE STATISTICS (Numerical Features)
           count         mean          std     min     25%     50%   

In [42]:
# Remove Duplicates
initial_rows = df.shape[0]
df.drop_duplicates(inplace=True)

# Handle Missing Values
if df.isnull().sum().any():
    for col in NUMERICAL_FEATURES:
        df[col].fillna(df[col].median(), inplace=True)
    for col in BINARY_FEATURES + NOMINAL_FEATURES:
        df[col].fillna(df[col].mode()[0], inplace=True)

#Outlier Treatment
for col in [TARGET, 'area']:
    upper_bound = df[col].quantile(0.99)
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

# Binary Encoding for Binary Features
df[BINARY_FEATURES] = df[BINARY_FEATURES].replace({'yes': 1, 'no': 0})

# One-Hot Encoding for Nominal Features
df_encoded = pd.get_dummies(df, columns=NOMINAL_FEATURES, drop_first=True, dtype=int)

# Identify the columns that need scaling (numerical, excluding the target)
features_to_scale = [col for col in df_encoded.columns if col in NUMERICAL_FEATURES and col != TARGET]

# Scaling Numerical Features (Min-Max Scaling)
scaler = MinMaxScaler()

# Fit and transform the features, converting back to a DataFrame
df_encoded[features_to_scale] = scaler.fit_transform(df_encoded[features_to_scale])

C:\Users\gusta\AppData\Local\Temp\ipykernel_18196\1610672577.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[BINARY_FEATURES] = df[BINARY_FEATURES].replace({'yes': 1, 'no': 0})


In [43]:
df_encoded.to_csv("../data/processed_housing.csv", index=False)